# Before You Start

The current set of notebooks are under constant development.

## Update Tutorial Repository

If you have previously cloned the tutorial repository, you may need to get the latest versions of the notebooks.

First check the status of your repository:
```
cd hls4ml-tutorial
git status 
```

You may have some _modified_ notebooks. For example:

```
# On branch csee-e6868-spring2021
# Changes not staged for commit:
#   (use "git add <file>..." to update what will be committed)
#   (use "git checkout -- <file>..." to discard changes in working directory)
#
#	modified:   part1_getting_started.ipynb
#	modified:   part2_advanced_config.ipynb
#
no changes added to commit (use "git add" and/or "git commit -a")
```

You can make a copy of those modified notebooks if you had significat changes, otherwise the easiest thing to do is to discard those changes.

**ATTENTION** You will loose your local changes!

```
git checkout *.ipynb
```

At this point, you can update you copy of the repository:
```
git pull
```

# Part 3: Compression

The terms _sparsity_, _pruning_, and _compression_ of a neural network are correlated and synonyms in this notebook. For more details look at this blog posts [[1]](https://numenta.com/blog/2019/08/30/case-for-sparsity-in-neural-networks-part-1-pruning) and [[2]](https://numenta.com/blog/2020/10/30/case-for-sparsity-in-neural-networks-part-2-dynamic-sparsity).

A sparse neural network is network where many of the weights are `0` and pruning is a compression technique to reduce the size of the neural network by _introducing more 0s_.

<img src="images/pruning.png" style="width:640px;" />



## Setup

As we did in the previous notebooks, let's import the libraries, call the magic functions, and setup the environment variables.

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os
os.environ['PATH'] = '/opt/xilinx/Vivado/2019.1/bin:' + os.environ['PATH']
def is_tool(name):
    from distutils.spawn import find_executable
    return find_executable(name) is not None

print('-----------------------------------')
if not is_tool('vivado_hls'):
    print('Xilinx Vivado HLS is NOT in the PATH')
else:
    print('Xilinx Vivado HLS is in the PATH')
print('-----------------------------------')

## Fetch the jet tagging dataset from Open ML

The [jet tagging dataset](https://www.openml.org/d/42468) is publicly available on [OpenML](https://www.openml.org).

In [ ]:
X_train_val = np.load('X_train_val.npy')
X_test = np.load('X_test.npy')
y_train_val = np.load('y_train_val.npy')
y_test = np.load('y_test.npy')
classes = np.load('classes.npy', allow_pickle=True)

## Now construct a model

We'll use the same architecture as in the previous notebooks: 3 hidden layers with 64, then 32, then 32 neurons. Each layer will use `relu` activation. Add an output layer with 5 neurons (one for each class), then finish with Softmax activation.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(16,), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu1'))
model.add(Dense(32, name='fc2', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu2'))
model.add(Dense(32, name='fc3', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu3'))
model.add(Dense(5, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='softmax', name='softmax'))

Let's visualize the model.

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

## Why do we need to compress a model?

Either you are running the model in software, as an FPGA firmware, or implemented as ASIC, a **smaller** model that retains the accuracy is always better. In some cases, you may be also willing to trade-off in accuracy for a smaller footprint, as we already have seen with _post-training quantization_.

<img src="images/before_compression.png" style="width:640px;" />

There is a lot in terms of papers and tools on the topic of pruning. For TensorFlow, you can have a look at this more  [comprehensive_guide](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide).

<img src="images/after_compression.png" style="width:800px;" />

## Train sparse

This time we'll use the Tensorflow model optimization sparsity to train a sparse model (forcing many weights to '0'). In this instance, the target sparsity is 75%.

In particular we use the [prune_low_magnitude](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/prune_low_magnitude) function. This function wraps a `tf.keras` model or layer with pruning functionality which sparsifies the layer's weights during training. For example, using this with 75% sparsity will ensure that 75% of the layer's weights are zero.

Among the parameters for the `prune_low_magnitude` function, we specify `PruningSchedule` object to control the pruning rate throughout training.
- In our case, we are using `ConstantSparsity`, which means that we are using constant sparsity (%) throughout training.
- You can also use `PolynomialDecay` based sparsity, in tha case more or fewer sparsity can be used with increasing or decreasing speed, as training progresses.

In [ ]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

pruning_params = {"pruning_schedule" : pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)

## Train the model
We'll use the same settings as the model for part 1: Adam optimizer with categorical crossentropy loss.
The callbacks will decay the learning rate and save the model into a directory 'model_2'
The model isn't very complex, so this should just take a few minutes even on the CPU.
If you've restarted the notebook kernel after training once, set `train = False` to load the trained model rather than training again.

In [ ]:
train = True
if train:
    adam = Adam(lr=0.0001)
    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    callbacks = all_callbacks(stop_patience = 1000,
                              lr_factor = 0.5,
                              lr_patience = 10,
                              lr_epsilon = 0.000001,
                              lr_cooldown = 2,
                              lr_minimum = 0.0000001,
                              outputDir = 'model_2')
    callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
    model.fit(X_train_val, y_train_val, batch_size=1024,
              epochs=30, validation_split=0.25, shuffle=True,
              callbacks = callbacks.callbacks)
    model.save('model_2/KERAS_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    model = load_model('model_2/KERAS_check_best_model.h5')

## Check sparsity
Make a quick check that the model was indeed trained sparse. We'll just make a histogram of the weights of the 1st layer, and hopefully observe a large peak in the bin containing '0'. Note logarithmic y axis.

In [ ]:
w = model.layers[0].weights[0].numpy()
h, b = np.histogram(w, bins=100)
plt.figure(figsize=(7,7))
plt.bar(b[:-1], h, width=b[1]-b[0])
plt.semilogy()
print('% of zeros = {}'.format(np.sum(w==0)/np.size(w)))

## Check performance

How does this 75% sparse model compare against the unpruned model?

Let's report the accuracy and make a ROC curve. The pruned model is shown with solid lines, the unpruned model from [Part 1](part1_getting_started.ipynb) is shown with dashed lines.

**Make sure you've trained the model from Part 1**

In [ ]:
import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
model_ref = load_model('model_1/KERAS_check_best_model.h5')

y_ref = model_ref.predict(X_test)
y_prune = model.predict(X_test)

print('-----------------------------------')
print("Unpruned Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_ref, axis=1))))
print("Pruned   Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_prune, axis=1))))
print('-----------------------------------')

# Enable logarithmic scale on TPR and FPR axes 
logscale_tpr = False # Y axis
logscale_fpr = False # X axis

fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.plotMultiClassRoc(y_test, y_ref, classes, logscale_tpr=logscale_tpr, logscale_fpr=logscale_fpr)
plt.gca().set_prop_cycle(None) # reset the colors
_ = plotting.plotMultiClassRoc(y_test, y_prune, classes, logscale_tpr=logscale_tpr, logscale_fpr=logscale_fpr, linestyle='--')

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], ls='-'),
         Line2D([0], [0], ls='--')]
from matplotlib.legend import Legend
leg = Legend(ax, lines, labels=['unpruned', 'pruned'],
            loc='center right', frameon=False)
_ = ax.add_artist(leg)

# Convert the model to FPGA firmware with hls4ml
Let's use the default configuration: `ap_fixed<16,6>` precision everywhere and `ReuseFactor=1`, so we can compare with the [Part 1](part1_getting_started.ipynb) model.

We need to use the function [strip_pruning](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/strip_pruning) to change the layer types back to their originals: once a model has been pruned to required sparsity, this method can be used to restore the original model with the sparse weights.

**This takes approx. 10 minutes on Columbia servers.**

While the C-Synthesis is running, we can monitor the progress looking at the log file by opening a terminal from the notebook home, and executing:

`tail -f model_2/hls4ml_prj/vivado_hls.log`

In [ ]:
%%time 
import hls4ml

# Save the model again but with the pruning 'stripped' to use the regular layer types
model = strip_pruning(model)

# Use the default configuration for hls4ml
config = hls4ml.utils.config_from_keras_model(model, granularity='model')
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='model_2/hls4ml_prj',
                                                       #part='xczu7ev-ffvc1156-2-e') # ZCU106
                                                       part='xczu3eg-sbva484-1-e') # Ultra96
                                                       #part='xc7z020clg400-1') # Pynq-Z1
                                                       #part='xc7z007sclg225-1') # MiniZed
hls_model.compile()

hls_results = hls_model.build(csim=False)

# print('-----------------------------------')
# #print(hls_results) # Print hashmap
# print("Estimated Clock Period: {} ns".format(hls_results['EstimatedClockPeriod']))
# print("Best/Worst Latency:     {} / {}".format(hls_results['BestLatency'], hls_results['WorstLatency']))
# print("Interval Min/Max:       {} / {}".format(hls_results['IntervalMin'], hls_results['IntervalMax']))
# print("BRAM_18K:               {} (Aval. {})".format(hls_results['BRAM_18K'], hls_results['AvailableBRAM_18K']))
# print("DSP48E:                 {} (Aval. {})".format(hls_results['DSP48E'], hls_results['AvailableDSP48E']))
# print("FF:                     {} (Aval. {})".format(hls_results['FF'], hls_results['AvailableFF']))
# print("LUT:                    {} (Aval. {})".format(hls_results['LUT'], hls_results['AvailableLUT']))
# print("URAM:                   {} (Aval. {})".format(hls_results['URAM'], hls_results['AvailableURAM']))
# print('-----------------------------------')

## Check the reports
Print out the reports generated by Vivado HLS. Pay attention to the Utilization Estimates' section in particular this time.

In [ ]:
hls4ml.report.read_vivado_report('model_2/hls4ml_prj/')

**ATTENTION: you need to have trained and synthesized the model from Part 1**

Print the report for the model trained in [Part 1](part1_getting_started.ipynb).

- Remember these two models have the same architecture, but the model in this notebook was trained using the sparsity API from TensorFlow Model Optimization.
- Notice how the resource usage had dramatically reduced (particularly the DSPs). When Vivado HLS notices an operation like `y = 0 * x` it can avoid placing a DSP for that operation. The impact of this is biggest when `ReuseFactor = 1`, but still applies at higher reuse as well. 

In [ ]:
hls4ml.report.read_vivado_report('model_1/hls4ml_prj')

Finally, remember how we are learning how to reduce the hardware-resource usage without affecting (too much) the model accuracy.

<img src="images/boards.png" style="width:640pt;" />